In [1]:
from datetime import datetime
from time import time
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin
from sklearn.base import BaseEstimator
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords  #pip install nltk
from nltk.corpus import stopwords
import nltk
nltk.download("stopwords")
from nltk.stem.porter import PorterStemmer
import string
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from gensim.models import word2vec  #pip install word2vec
from wordcloud import WordCloud  #pip install wordcloud
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
from sklearn.manifold import TSNE, MDS
from sklearn.decomposition import NMF, LatentDirichletAllocation
from textblob import TextBlob  #Sentiment Analysis - pip install textblob
from sklearn.decomposition import TruncatedSVD, NMF
import matplotlib.patches as mpatches
import matplotlib
path_to_csv = '../../../cs82_advanced_machine_learning_data/HW2/papers.csv'

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\khan_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [58]:
df_allPlays = pd.read_csv('Shakespeare_data.csv')
print(allPlays.shape)
df_allPlays.head()

(111396, 6)


Dataline      Play  PlayerLinenumber ActSceneLine         Player  \
0         1  Henry IV               NaN          NaN            NaN   
1         2  Henry IV               NaN          NaN            NaN   
2         3  Henry IV               NaN          NaN            NaN   
3         4  Henry IV               1.0        1.1.1  KING HENRY IV   
4         5  Henry IV               1.0        1.1.2  KING HENRY IV   

                                          PlayerLine  
0                                              ACT I  
1                       SCENE I. London. The palace.  
2  Enter KING HENRY, LORD JOHN OF LANCASTER, the ...  
3             So shaken as we are, so wan with care,  
4         Find we a time for frighted peace to pant,

In [59]:
df_allPlays.isna().sum()

Dataline               0
Play                   0
PlayerLinenumber       3
ActSceneLine        6243
Player                 7
PlayerLine             0
dtype: int64

In [61]:
df_allPlays[df_allPlays.isnull().any(axis=1)].head()

Dataline      Play  PlayerLinenumber ActSceneLine        Player  \
0           1  Henry IV               NaN          NaN           NaN   
1           2  Henry IV               NaN          NaN           NaN   
2           3  Henry IV               NaN          NaN           NaN   
111       112  Henry IV              10.0          NaN  WESTMORELAND   
112       113  Henry IV              10.0          NaN  WESTMORELAND   

                                            PlayerLine  
0                                                ACT I  
1                         SCENE I. London. The palace.  
2    Enter KING HENRY, LORD JOHN OF LANCASTER, the ...  
111                                             Exeunt  
112    SCENE II. London. An apartment of the Prince's.

In [62]:
df_allPlays.ActSceneLine.isnull().groupby([df_allPlays['Play']]).sum()

Play
A Comedy of Errors           88.0
A Midsummer nights dream    138.0
A Winters Tale              125.0
Alls well that ends well    158.0
Antony and Cleopatra        297.0
As you like it              145.0
Coriolanus                  224.0
Cymbeline                   201.0
Hamlet                      224.0
Henry IV                    161.0
Henry V                     162.0
Henry VI Part 1             206.0
Henry VI Part 2             204.0
Henry VI Part 3             199.0
Henry VIII                  140.0
Julius Caesar               174.0
King John                   117.0
King Lear                   266.0
Loves Labours Lost          108.0
Measure for measure         158.0
Merchant of Venice          137.0
Merry Wives of Windsor      217.0
Much Ado about nothing      120.0
Othello                     199.0
Pericles                    169.0
Richard II                  137.0
Richard III                 236.0
Romeo and Juliet            217.0
Taming of the Shrew         169.0
The Tempe

In [67]:
class RemoveAllNans (TransformerMixin, BaseEstimator):
    def __init__(self, prints = False):
        self.prints = prints
    def fit(self, X, y = None):
        return self
    def transform(self, X):
        return X.dropna()
    
    
class split_act_scene_line (TransformerMixin, BaseEstimator):
    def __init__(self, prints = False):
        self.prints = prints
    def fit(self, X, y = None):
        return self
    def transform(self, X):
        X[['Act','Scene','Line']] = X['ActSceneLine'].str.split('.', expand=True).apply(pd.to_numeric)
        return X
    
class group_data(TransformerMixin, BaseEstimator):
    def __init__(self, grouping = "Act",  prints = False):
        self.prints = prints
        self.grouping = grouping
        print(self.grouping)
    def fit(self, X, y = None):
        return self
    def transform(self, X):
        if self.grouping == "Play" : 
                group_by = ["Play"]
        elif self.grouping == "Act" : 
                group_by = ["Play", "Act"]
        elif self.grouping == "Scene":
                group_by = ["Play", "Act", "Scene"]

        lines = X.groupby(group_by)["PlayerLine"].sum().reset_index()["PlayerLine"]
        players = X.groupby(group_by)["Player"].unique().reset_index()
        X_new = pd.concat([players, lines], axis=1)
        return X_new

In [73]:
allplays_pipeline = Pipeline([
    ('romove_na', RemoveAllNans()),
    ('split_act_scene_line', split_act_scene_line()),
    ('group_data', group_data(grouping="Act"))
])

df_PlayData = allplays_pipeline.fit_transform(df_allPlays)

df_PlayData.head(200)

Act


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


Play  Act  \
0          A Comedy of Errors    1   
1          A Comedy of Errors    2   
2          A Comedy of Errors    3   
3          A Comedy of Errors    4   
4          A Comedy of Errors    5   
5    A Midsummer nights dream    1   
6    A Midsummer nights dream    2   
7    A Midsummer nights dream    3   
8    A Midsummer nights dream    4   
9    A Midsummer nights dream    5   
10             A Winters Tale    1   
11             A Winters Tale    2   
12             A Winters Tale    3   
13             A Winters Tale    4   
14             A Winters Tale    5   
15   Alls well that ends well    1   
16   Alls well that ends well    2   
17   Alls well that ends well    3   
18   Alls well that ends well    4   
19   Alls well that ends well    5   
20       Antony and Cleopatra    1   
21       Antony and Cleopatra    2   
22       Antony and Cleopatra    3   
23       Antony and Cleopatra    4   
24       Antony and Cleopatra    5   
25             As you like it    1   
26             As you like it    2   
27             As you like it    3   
28             As you like it    4   
29             As you like it    5   
..                        ...  ...   
151           Timon of Athens    1   
152           Timon of Athens    2   
153           Timon of Athens    3   
154           Timon of Athens    4   
155           Timon of Athens    5   
156          Titus Andronicus    1   
157          Titus Andronicus    2   
158          Titus Andronicus    3   
159          Titus Andronicus    4   
160          Titus Andronicus    5   
161      Troilus and Cressida    1   
162      Troilus and Cressida    2   
163      Troilus and Cressida    3   
164      Troilus and Cressida    4   
165      Troilus and Cressida    5   
166             Twelfth Night    1   
167             Twelfth Night    2   
168             Twelfth Night    3   
169             Twelfth Night    4   
170             Twelfth Night    5   
171   Two Gentlemen of Verona    1   
172   Two Gentlemen of Verona    2   
173   Two Gentlemen of Verona    3   
174   Two Gentlemen of Verona    4   
175   Two Gentlemen of Verona    5   
176                   macbeth    1   
177                   macbeth    2   
178                   macbeth    3   
179                   macbeth    4   
180                   macbeth    5   

                                                Player  \
0    [AEGEON, DUKE SOLINUS, Gaoler, First Merchant,...   
1    [ADRIANA, LUCIANA, DROMIO OF EPHESUS, OF SYRAC...   
2    [LUCIANA, OF EPHESUS, DROMIO OF EPHESUS, BALTH...   
3    [Second Merchant, ANGELO, Officer, OF EPHESUS,...   
4    [ANGELO, Second Merchant, OF SYRACUSE, ADRIANA...   
5    [THESEUS, HIPPOLYTA, EGEUS, HERMIA, DEMETRIUS,...   
6    [PUCK, Fairy, OBERON, TITANIA, DEMETRIUS, HELE...   
7    [BOTTOM, QUINCE, SNOUT, STARVELING, PUCK, FLUT...   
8    [PUCK, TITANIA, BOTTOM, PEASEBLOSSOM, COBWEB, ...   
9    [HIPPOLYTA, THESEUS, LYSANDER, PHILOSTRATE, Pr...   
10   [ARCHIDAMUS, CAMILLO, POLIXENES, LEONTES, HERM...   
11   [HERMIONE, First Lady, MAMILLIUS, Second Lady,...   
12   [CLEOMENES, DION, LEONTES, Officer, HERMIONE, ...   
13   [Shepherd, Time, POLIXENES, CAMILLO, AUTOLYCUS...   
14   [CLEOMENES, LEONTES, PAULINA, DION, Gentleman,...   
15   [COUNTESS, BERTRAM, LAFEU, HELENA, PAROLLES, P...   
16   [KING, First Lord, Second Lord, Both, PAROLLES...   
17   [DUKE, First Lord, Second Lord, COUNTESS, Clow...   
18   [Second Lord, First Soldier, PAROLLES, All, Se...   
19   [HELENA, Gentleman, Widow, PAROLLES, Clown, LA...   
20   [PHILO, CLEOPATRA, MARK ANTONY, Attendant, DEM...   
21   [POMPEY, MENECRATES, MENAS, VARRIUS, LEPIDUS, ...   
22   [VENTIDIUS, SILIUS, AGRIPPA, DOMITIUS ENOBARBU...   
23   [OCTAVIUS CAESAR, MECAENAS, MARK ANTONY, DOMIT...   
24   [OCTAVIUS CAESAR, DOLABELLA, DERCETAS, AGRIPPA...   
25   [ORLANDO, ADAM, OLIVER, DENNIS, CHARLES, CELIA...   
26   [DUKE SENIOR, AMIENS, First Lord, Second Lord,...   
27   [DUKE FREDERICK, OLIVER, ORLANDO, CORIN

In [74]:
print(df_PlayData.isna().sum())
print(df_PlayData.dtypes)
print(df_PlayData.shape)

Play          0
Act           0
Player        0
PlayerLine    0
dtype: int64
Play          object
Act            int64
Player        object
PlayerLine    object
dtype: object
(181, 4)
